In [1]:
from keras.applications.resnet50 import ResNet50 as CNN
#from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model
from keras.layers import Dense, Flatten
#from keras import backend as K

import pandas as pd
#import numpy as np
import bson
import os
#import io
#from tqdm import *

from utilities import utils
from utilities.generate_submit import generate_submit

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


In [2]:
data_dir = "inputs/"

test_bson_path = os.path.join(data_dir, "test.bson")
num_test_products = 1768172
#num_test_products = 1000

categories_df = pd.read_csv("inputs/categories.csv", index_col=0)
_, idx2cat = utils.make_category_tables(categories_df)

In [3]:
#submission_df = pd.read_csv(data_dir + "sample_submission.csv")
#submission_df.head()

In [4]:
#test_datagen = ImageDataGenerator() #ImageDataGenerator(preprocessing_function=preprocess_input)
data = bson.decode_file_iter(open(test_bson_path, "rb"))

In [5]:
crop_size = 197
num_classes = 5270

model = CNN(include_top=False, input_shape=(crop_size, crop_size, 3), weights=None)
x = Flatten()(model.output)
classifier = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=classifier)

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.load_weights("weights/ResNet50-2017-10-10-0014.hdf5")

In [ ]:
generate_submit(model, data, 224, 197, num_test_products, idx2cat, 
                'outputs/submission05','outputs/predictions05', q_size=200, out_q_size=100)

  0%|          | 0/1768172 [00:00<?, ?it/s]

Start predicting on samples...


  1%|          | 11435/1768172 [10:19<32:16:48, 15.12it/s]

submission_df.to_csv("outputs/my_submission.csv.gz", compression="gzip", index=False)